## Accompanying code to 'Uncertainty Risk Parity' https://ssrn.com/abstract=3406321
### Example of uncertain risk parity and uncertain risk contributions

In [1]:
import numpy as np
import lib_risk_parity as rp

# 0 - Get estimates - C, covC, (to check out of sample) Ctest

In [2]:
import pandas as pd

fname = 'etf_weekly_rets.csv'
retsdf = pd.read_csv(fname, header=0, index_col=0) # (n x T) cols are old -> new
ids = retsdf.index
rets = np.matrix(retsdf)
del retsdf

def est_cov_of_cov_disjoint_periods(r, numsets=10):
    # function, in ad-hoc fashion, estimates covariance of covariance
    # as covariance of covariance estimates from disjoint periods
    # r = (n x T) returns
    # numsets = (int) number of sets to break r into
    # returns (n x n x n x n) covariance of covariance entries
    n, T = r.shape
    l = int(np.floor(T/numsets))  # num periods in each set
    M = np.zeros((n**2, numsets)) # holds each sets cov as a vector
    for i in range(numsets):
        M[:,i] = np.cov(r[:,i*l:(i+1)*l]).reshape([-1])  # store covariance over interval as a vector
    return np.cov(M).reshape((n,n,n,n))  # take cov of cov estimates, reshape from (n^2 x n^2) to (n x n x n x n)

def get_estimates(r, train_fraction=0.75, numsets_for_covcov=10):
    # function estimates train cov, train cov of cov, and test cov of n series
    # r = (n x T) n series of length T
    # train_fraction = fraction of data used for training
    # numsets_for_covcov = number of sets to break training set into to estimate cov of cov
    # returns    C = (n x n) estimated cov over training portion
    #         covC = (n x n x n x n) estimated cov of cov over training portion
    #        Ctest = (n x n) estimated cov over test portion
    n, T = r.shape
    # split into train and test
    T0 = int(np.ceil(0.75*T))
    Ctest = np.cov(r[:,T0:]) # (n x n) cov est on test set
    C = np.cov(r[:,:T0]) # (n x n) cov est on train set
    covC = est_cov_of_cov_disjoint_periods(r=r[:,:T0], numsets=numsets_for_covcov) # (n x n x n x n) cov of cov est on train set
    return C, covC, Ctest

C, covC, Ctest = get_estimates(r=rets, train_fraction=0.75, numsets_for_covcov=12)

def cov2corr(C):
    # converts covariance matrix to correlation matrix
    # C = (N x N) covariance matrix
    # returns (N x N) correlation matrix
    _invsd = 1./np.diag(C)**0.5
    return (C * _invsd).T * _invsd

def print_sd_cor(C, d=2):
    print('sd:', np.around(np.diag(C), d))
    print('cor:\n', np.around(cov2corr(C), d))

print('training set')
print_sd_cor(C)
print('\ntest set')
print_sd_cor(Ctest)

training set
sd: [0.17 6.35 2.98 3.33]
cor:
 [[ 1.   -0.19 -0.19 -0.19]
 [-0.19  1.    0.89  0.78]
 [-0.19  0.89  1.    0.87]
 [-0.19  0.78  0.87  1.  ]]

test set
sd: [ 3.14 23.81 20.96 17.96]
cor:
 [[1.   0.69 0.74 0.53]
 [0.69 1.   0.93 0.85]
 [0.74 0.93 1.   0.89]
 [0.53 0.85 0.89 1.  ]]


# 1 - Construct portfolios

In [3]:
n = C.shape[0]
target = np.full(n, 1./n)  # target is equal risk contributions

In [4]:
w0 = rp.std_risk_parity(target=target, C=C)
w = rp.uncertain_risk_parity(target=target, C=C, covC=covC)

# 2 - Results

In [5]:
def print_results(ids, w, Ctest, C, covC=None):    
    M = np.zeros((4,n))
    M[0,:] = w
    M[1,:] = w * C.dot(w)     # variance contributions on train set
    M[3,:] = w * Ctest.dot(w) # variance contributions on test set
    if covC is not None:
        covv = np.einsum('i,k,ijkl',w,w,covC)  # cov of variance contributions on train set
        M[2,:] = np.diag(covv) ** 0.5  # standard deviation of variance contributiosn
    df = pd.DataFrame(np.around(M,3), columns=ids)
    df.index=('weight', 'v_train', '+/-','v_test')
    return df

In [6]:
print('std risk parity')
print_results(ids, w0, Ctest, C)

std risk parity


,AGG,EEM,EFA,SPY
weight,0.748,0.066,0.094,0.092
v_train,0.064,0.064,0.064,0.064
+/-,0.000,0.000,0.000,0.000
v_test,2.740,0.636,0.883,0.682


In [7]:
print('uncertain risk parity')
print_results(ids, w, Ctest, C, covC=covC)

uncertain risk parity


,AGG,EEM,EFA,SPY
weight,0.790,0.059,0.083,0.068
v_train,0.078,0.047,0.046,0.038
+/-,0.080,0.669,0.542,0.607
v_test,2.838,0.534,0.735,0.467
